# Overview
Sometimes we make decisions beyond the rating of a restaurant. For example, if a restaurant has a high rating but it often fails to pass hygiene inspections, then this information can dissuade many people to eat there. Using this hygiene information could lead to a more informative system; however, it is often the case where we don’t have such information for all the restaurants, and we are left to make predictions based on the small sample of data points.

In this task, you are going to predict whether a set of restaurants will pass the public health inspection tests given the corresponding Yelp text reviews along with some additional information such as the locations and cuisines offered in these restaurants. Making a prediction about an unobserved attribute using data mining techniques represents a wide range of important applications of data mining. Through working on this task, you will gain direct experience with such an application. Due to the flexibility of using as many indicators for prediction as possible, this would also give you an opportunity to potentially combine many different algorithms you have learned from the courses in the Data Mining Specialization to solve a real world problem and experiment with different methods to understand what’s the most effective way of solving the problem.

## About the Dataset
You should first download the [dataset](https://d396qusza40orc.cloudfront.net/dataminingcapstone/Task6/Hygiene.tar.gz). The dataset is composed of a training subset containing 546 restaurants used for training your classifier, in addition to a testing subset of 12753 restaurants used for evaluating the performance of the classifier. In the training subset, you will be provided with a binary label for each restaurant, which indicates whether the restaurant has passed the latest public health inspection test or not, whereas for the testing subset, you will not have access to any labels. The dataset is spread across three files such that the first 546 lines in each file corresponding to the training subset, and the rest are part of the testing subset. Below is a description of each file:

* **hygiene.dat**: Each line contains the concatenated text reviews of one restaurant.  
* **hygiene.dat.labels**: For the first 546 lines, a binary label (0 or 1) is used where a 0 indicates that the restaurant has passed the latest public health inspection test, while a 1 means that the restaurant has failed the test. The rest of the lines have "[None]" in their label field implying that they are part of the testing subset.
* **hygiene.dat.additional**: It is a CSV (Comma-Separated Values) file where the first value is a list containing the cuisines offered, the second value is the zip code, which gives an idea about the location, the third is the number of reviews, and the fourth is the average rating, which can vary between 0 and 5 (5 being the best).

For testing, we use the F1 measure, which is the harmonic mean of precision and recall, to rank the submissions in the leaderboard. The F1 measure will be based on the macro-averages of precision and recall (macro-averaging is used here to ensure that the two classes are given equal weight as we do not want class 0 to dominate the measure).

In [1]:
hygiene_text_path= "./Hygiene/hygiene.dat"
hygiene_labels_path= "./Hygiene/hygiene.dat.labels"
hygiene_additional_path= "./Hygiene/hygiene.dat.additional"

In [165]:
import pandas as pd
import numpy as np
import multiprocessing
import gensim
import nltk
import spacy

from UtilWordEmbedding import MeanEmbeddingVectorizer

from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin


from gensim.parsing.preprocessing import strip_tags, strip_punctuation, strip_numeric, strip_short
from gensim.parsing.preprocessing import strip_multiple_whitespaces, strip_non_alphanum, remove_stopwords, stem_text
from gensim.models.word2vec import Word2Vec
from nltk.stem import WordNetLemmatizer, SnowballStemmer

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction import DictVectorizer

from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn import svm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, GridSearchCV
from tqdm import tqdm


from nltk.corpus import stopwords 
STOP_WORDS = set(stopwords.words('english'))


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
SEED=26

To Download Glove Embeddings
```
wget http://nlp.stanford.edu/data/glove.6B.zip
unzip glove.6B.zip
```

## 1. Data Prep
Preprocessing Steps:
* tokenization and cleaning
* stopword removal
* stemming and lemmatization

In [3]:
# tokenize and preprocess
# https://radimrehurek.com/gensim/parsing/preprocessing.html
FILTERS_LIST = [lambda x: x.lower(), # lowercase  
                strip_tags, # remove tags
                strip_punctuation, # replace punctuation characters with spaces
                strip_multiple_whitespaces, # remove repeating whitespaces
                # strip_numeric, # remove numbers
                gensim.parsing.preprocessing.remove_stopwords, # remove stopwords
                strip_short, # remove words less than minsize=3 characters long]
                stem_text]
def preprocess(text):
    """
    strip_tags, strip_punctuation, strip_multiple_whitespaces, strip_numeric, 
    """
    result_stemmed = []
    for token in gensim.parsing.preprocessing.preprocess_string(text, FILTERS_LIST):
        result_stemmed.append(WordNetLemmatizer().lemmatize(token))
    return result_stemmed

In [4]:
%%time
texts = []
preprocessed_texts = []

with open(hygiene_text_path) as f:
    texts = f.readlines()
    
for _text in tqdm(texts):
    result_stemmed = preprocess(_text)
    preprocessed_texts.append(result_stemmed)
    
all_preprocessed_texts = [" ".join(_text) for _text in preprocessed_texts]

100%|██████████| 13299/13299 [01:33<00:00, 142.81it/s]


CPU times: user 1min 32s, sys: 732 ms, total: 1min 33s
Wall time: 1min 33s


In [33]:
N = 546

# labels 
with open(hygiene_labels_path, 'r') as f:
    labels = [l.rstrip() for l in f]

# texts = []
# with open(hygiene_text_path, 'r') as f:
#     texts = f.read().splitlines(True)


df = pd.DataFrame({"label":labels, "text": texts, 
                   "preprocessed_texts": all_preprocessed_texts,
                   "tokenized_texts": preprocessed_texts})
hygiene_additional = pd.read_csv(hygiene_additional_path,  
                                 names=["cuisines_offered", "zipcode", "num_reviews", "avg_rating"],
                                 dtype={"cuisines_offered": str, 
                                        "zipcode": str,
                                        "num_reviews": str})
df = df.join(hygiene_additional)
df['avg_rating'] = df['avg_rating'].apply(lambda x: str(int(round(x, 0))))

print(df.info())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13299 entries, 0 to 13298
Data columns (total 8 columns):
label                 13299 non-null object
text                  13299 non-null object
preprocessed_texts    13299 non-null object
tokenized_texts       13299 non-null object
cuisines_offered      13299 non-null object
zipcode               13299 non-null object
num_reviews           13299 non-null object
avg_rating            13299 non-null object
dtypes: object(8)
memory usage: 831.3+ KB
None


,label,text,preprocessed_texts,tokenized_texts,cuisines_offered,zipcode,num_reviews,avg_rating
0,1,"The baguettes and rolls are excellent, and alt...",baguett roll excel haven tri excit dozen plu t...,"[baguett, roll, excel, haven, tri, excit, doze...","['Vietnamese', 'Sandwiches', 'Restaurants']",98118,4,4
1,1,I live up the street from Betty. &#160;When my...,live street betti 160 sister town spring break...,"[live, street, betti, 160, sister, town, sprin...","['American (New)', 'Restaurants']",98109,21,4
2,1,I'm worried about how I will review this place...,worri review place strongli think bad night pl...,"[worri, review, place, strongli, think, bad, n...","['Mexican', 'Restaurants']",98103,14,3
3,0,Why can't you access them on Google street vie...,access googl street view like medina yarrow po...,"[access, googl, street, view, like, medina, ya...","['Mexican', 'Tex-Mex', 'Restaurants']",98112,42,4
4,0,Things to like about this place: homemade guac...,thing like place homemad guacamol varieti tast...,"[thing, like, place, homemad, guacamol, variet...","['Mexican', 'Restaurants']",98102,12,3


In [46]:
%%time
train_df = df[df["label"] != "[None]"]
test_df = df[df["label"] == "[None]"]

additional_feats = ["cuisines_offered", "zipcode", "num_reviews", "avg_rating"]

train = train_df[["text"] + additional_feats]
train_preprocessed = train_df[["preprocessed_texts"] + additional_feats]
train_tokenized = train_df[["tokenized_texts"] + additional_feats]
train_labels = train_df["label"].astype(int) # needed by sklearn

test = test_df[["text"] + additional_feats]
test_preprocessed = test_df[["preprocessed_texts"] + additional_feats]
test_tokenized = test_df[["tokenized_texts"] + additional_feats]
test_labels = test_df["label"]

print(train.shape, train_preprocessed.shape, train_tokenized.shape, train_labels.shape)
print(test.shape, test_preprocessed.shape, test_tokenized.shape, test_labels.shape)
print(train.dtypes, train_preprocessed.dtypes, train_tokenized.dtypes)

(546, 5) (546, 5) (546, 5) (546,)
(12753, 5) (12753, 5) (12753, 5) (12753,)
text                object
cuisines_offered    object
zipcode             object
num_reviews         object
avg_rating          object
dtype: object preprocessed_texts    object
cuisines_offered      object
zipcode               object
num_reviews           object
avg_rating            object
dtype: object tokenized_texts     object
cuisines_offered    object
zipcode             object
num_reviews         object
avg_rating          object
dtype: object
CPU times: user 16.1 ms, sys: 1.95 ms, total: 18.1 ms
Wall time: 16.3 ms


In [47]:
display(train.head())
display(train_preprocessed.head())
display(train_tokenized.head())

,text,cuisines_offered,zipcode,num_reviews,avg_rating
0,"The baguettes and rolls are excellent, and alt...","['Vietnamese', 'Sandwiches', 'Restaurants']",98118,4,4
1,I live up the street from Betty. &#160;When my...,"['American (New)', 'Restaurants']",98109,21,4
2,I'm worried about how I will review this place...,"['Mexican', 'Restaurants']",98103,14,3
3,Why can't you access them on Google street vie...,"['Mexican', 'Tex-Mex', 'Restaurants']",98112,42,4
4,Things to like about this place: homemade guac...,"['Mexican', 'Restaurants']",98102,12,3


,preprocessed_texts,cuisines_offered,zipcode,num_reviews,avg_rating
0,baguett roll excel haven tri excit dozen plu t...,"['Vietnamese', 'Sandwiches', 'Restaurants']",98118,4,4
1,live street betti 160 sister town spring break...,"['American (New)', 'Restaurants']",98109,21,4
2,worri review place strongli think bad night pl...,"['Mexican', 'Restaurants']",98103,14,3
3,access googl street view like medina yarrow po...,"['Mexican', 'Tex-Mex', 'Restaurants']",98112,42,4
4,thing like place homemad guacamol varieti tast...,"['Mexican', 'Restaurants']",98102,12,3


,tokenized_texts,cuisines_offered,zipcode,num_reviews,avg_rating
0,"[baguett, roll, excel, haven, tri, excit, doze...","['Vietnamese', 'Sandwiches', 'Restaurants']",98118,4,4
1,"[live, street, betti, 160, sister, town, sprin...","['American (New)', 'Restaurants']",98109,21,4
2,"[worri, review, place, strongli, think, bad, n...","['Mexican', 'Restaurants']",98103,14,3
3,"[access, googl, street, view, like, medina, ya...","['Mexican', 'Tex-Mex', 'Restaurants']",98112,42,4
4,"[thing, like, place, homemad, guacamol, variet...","['Mexican', 'Restaurants']",98102,12,3


In [ ]:
# # just use cross_val_score
# X_train, X_test, y_train, y_test = train_test_split(train_preprocessed, train_labels, test_size= 0.2, random_state=SEED)

## 2. Model Experiments

Models:  
* Naive Bayes
* SVM
* Logistic Regression
* Random Forest
* XGBoost

Feature Engineering:  
* Count Vectorizer
* Tfidf Vectorizer
* Previously Generated Embeddings
    * word2vec
    * fasttext

### Scratch

In [330]:
%%time
from sklearn import preprocessing

pipeline = Pipeline([
    ('preprocess', ColumnTransformer(
        [('cuisines_offered', CountVectorizer(min_df=10), 'cuisines_offered'),
         ('zipcode', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['zipcode']),
         ('num_reviews', CountVectorizer(max_df=7, token_pattern='\d+'), 'num_reviews'),
         ('avg_rating', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['avg_rating']),
         ('text', TfidfVectorizer(
                    stop_words='english',
                    strip_accents='unicode',
                    min_df=3,
                    max_df=0.5,
                    ngram_range=(1, 3),
                    max_features=500), 'preprocessed_texts')],
        remainder='passthrough',
    )),
    ('clf', MultinomialNB())
], verbose=False)

# pipeline.fit(X_train, y_train)
# y_pred = pipeline.predict(X_test)
# scores = metrics.f1_score(y_test, y_pred)
scores = cross_val_score(pipeline, train_preprocessed, train_labels, cv=5, scoring= 'f1_macro')
print(scores)
print("Average F1-Score: %0.5f" % np.average(scores))

[0.62650104 0.72474747 0.67792317 0.68783693 0.60507246]
Average F1-Score: 0.66442
CPU times: user 12.4 s, sys: 327 ms, total: 12.7 s
Wall time: 12.8 s


In [283]:
%%time
from sklearn import preprocessing

pipeline = Pipeline([
    ('preprocess', ColumnTransformer(
        [('cuisines_offered', CountVectorizer(min_df=10), 'cuisines_offered'),
         ('zipcode', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['zipcode']),
         ('num_reviews', CountVectorizer(max_df=7, token_pattern='\d+'), 'num_reviews'),
         ('avg_rating', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['avg_rating']),
         ('text', TfidfVectorizer(
                    stop_words='english',
                    strip_accents='unicode',
                    min_df=3,
                    max_df=0.5,
                    ngram_range=(1, 3),
                    max_features=500), 'text')],
        remainder='passthrough',
    )),
    ('clf', MultinomialNB())
], verbose=False)

# pipeline.fit(X_train, y_train)
# y_pred = pipeline.predict(X_test)
# scores = metrics.f1_score(y_test, y_pred)
scores = cross_val_score(pipeline, train, train_labels, cv=5, scoring= 'f1')
print(scores)
print("Average F1-Score: %0.5f" % np.average(scores))

[0.62857143 0.72222222 0.68965517 0.66037736 0.54347826]
Average F1-Score: 0.64886
CPU times: user 14.2 s, sys: 360 ms, total: 14.5 s
Wall time: 14.5 s


In [275]:
print(len(df['num_reviews'].value_counts()))
df['num_reviews'].value_counts()

158


1      2193
2      1480
3      1126
4       934
5       767
6       675
7       600
8       484
9       458
10      403
11      352
12      315
13      268
14      246
16      200
15      199
17      185
18      160
19      131
20      125
21      124
22      124
23      107
24      101
25       97
28       86
26       81
27       77
29       68
30       60
32       52
33       51
37       44
36       44
31       43
34       43
35       42
39       39
44       37
38       33
43       30
47       28
42       27
40       26
45       25
46       25
41       23
54       22
59       20
49       15
52       15
63       15
55       15
62       14
48       14
51       14
73       13
57       12
61       11
50       11
56       11
53       10
58       10
66        9
78        9
89        8
83        8
67        8
60        8
65        7
70        6
64        6
93        6
77        6
69        6
84        5
85        5
112       5
76        5
100       4
91        4
74        4
68        4
80  

In [277]:
print(len(df['cuisines_offered'].value_counts()))
df['cuisines_offered'].value_counts()

388


['Thai', 'Restaurants']                                                          640
['American (New)', 'Restaurants']                                                596
['American (Traditional)', 'Restaurants']                                        589
['Mexican', 'Restaurants']                                                       572
['Pizza', 'Restaurants']                                                         524
['Vietnamese', 'Restaurants']                                                    465
['Japanese', 'Sushi Bars', 'Restaurants']                                        459
['Sandwiches', 'Restaurants']                                                    430
['Chinese', 'Restaurants']                                                       394
['Italian', 'Pizza', 'Restaurants']                                              327
['Japanese', 'Restaurants']                                                      282
['Italian', 'Restaurants']                                       

In [276]:
print(len(df['avg_rating'].value_counts()))
print(len(df['zipcode'].value_counts()))

5
30


In [198]:
%%time
pipeline = Pipeline([
    ('preprocess', ColumnTransformer(
        [('cuisines_offered', CountVectorizer(), 'cuisines_offered'),
         ('zipcode', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['zipcode']),
         ('num_reviews', CountVectorizer(token_pattern='\d+'), 'num_reviews'),
         ('avg_rating', CountVectorizer(token_pattern='\d+'), 'avg_rating'),
         ('text', TfidfVectorizer(
                    stop_words='english',
                    strip_accents='unicode',
                    min_df=3,
                    max_df=0.5,
                    ngram_range=(1, 3),
                    max_features=500), 'preprocessed_texts')],
        remainder='passthrough',
    )),
    ('clf', MultinomialNB())
], verbose=False)

# pipeline.fit(X_train, y_train)
# y_pred = pipeline.predict(X_test)
# scores = metrics.f1_score(y_test, y_pred)
scores = cross_val_score(pipeline, train_preprocessed, train_labels, cv=5, scoring= 'f1')
print(scores)
print("Average F1-Score: %0.5f" % np.average(scores))

[0.63551402 0.7027027  0.65486726 0.63636364 0.52272727]
Average F1-Score: 0.63043
CPU times: user 11.5 s, sys: 420 ms, total: 12 s
Wall time: 12 s


In [ ]:
%%time
pipeline = Pipeline([
    ('union', ColumnTransformer(
        [('cuisines_offered', CountVectorizer(), 'cuisines_offered'),
         ('zipcode', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['zipcode']),
         ('num_reviews', CountVectorizer(token_pattern='\d+'), 'num_reviews'),
         ('avg_rating', CountVectorizer(token_pattern='\d+'), 'avg_rating'),
         ('text', TfidfVectorizer(
                stop_words='english',
                strip_accents='unicode',
                min_df=15,
                max_df=0.5,
                ngram_range=(1, 3),
                max_features=500), 'text')],
        remainder='passthrough',
    )),
    ('clf', MultinomialNB())
], verbose=False)

# pipeline.fit(X_train, y_train)
# y_pred = pipeline.predict(X_test)
# score = metrics.f1_score(y_test, y_pred)
scores = cross_val_score(pipeline, train, train_labels, cv=5, scoring= 'f1')
print(scores)
print("Average F1-Score: %0.5f" % np.average(scores))

### Create Function for Testing

In [320]:
%%time
def test_classifier(clf, X, y, vectorizer, text_col='text'):
    pipeline = Pipeline([
        ('union', ColumnTransformer(
        [('cuisines_offered', CountVectorizer(min_df=10), 'cuisines_offered'),
         ('zipcode', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['zipcode']),
         ('num_reviews', CountVectorizer(max_df=7, token_pattern='\d+'), 'num_reviews'),
         ('avg_rating', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['avg_rating']),
         ('text', vectorizer, text_col)],
        remainder='passthrough',
    )),
        ('clf', clf)
    ], verbose=False)
    scores = cross_val_score(pipeline, X, y, cv=5, scoring= 'f1_macro')
    print(clf)
    print(scores)
    cv_score = np.average(scores)
    return cv_score

CPU times: user 7 µs, sys: 7 µs, total: 14 µs
Wall time: 16.7 µs


In [322]:
classifiers = {
    'Naive Bayes': MultinomialNB(),
    'Support Vector Machine': svm.SVC(),
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(random_state=SEED, n_estimators=500, n_jobs=-1),
    #'Gradient Boosting': GradientBoostingClassifier()
    'XGBoost': XGBClassifier(n_estimators=500, 
                            max_depth=5, 
                            learning_rate=0.2, 
                            objective='binary:logistic',
                            scale_pos_weight=2,
                            n_jobs=-1,
                            random_state=SEED)
}

tfidf = TfidfVectorizer(
                    stop_words='english',
                    strip_accents='unicode',
                    min_df=3,
                    max_df=0.5,
                    ngram_range=(1, 3),
                    max_features=500)
bow = CountVectorizer(
                stop_words=STOP_WORDS,
                strip_accents='unicode',
                min_df=15,
                max_df=0.5,
                ngram_range=(1, 3))

### 2.1 BOW - No Preprocessing

In [323]:
%%time
for clf_name, clf in classifiers.items():
    cv_score = test_classifier(clf, train, train_labels, 
                               vectorizer=bow, text_col='text')
    print('{}: {}'.format(clf_name, cv_score))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
[0.59880637 0.72456199 0.67879094 0.64220183 0.60911885]
Naive Bayes: 0.650695997690299
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
[0.61818813 0.57806452 0.58209082 0.59444493 0.54821774]
Support Vector Machine: 0.5842012268003629


/anaconda3/envs/uiuc/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/anaconda3/envs/uiuc/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
[0.57269196 0.58365164 0.60497261 0.63274933 0.6292517 ]
Logistic Regression: 0.6046634471652974
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=26, verbose=0,
                       warm_start=False)
[0.6979782  0.58365164 0.58492003 

### 2.2 BOW - Preprocessing

In [324]:
%%time
for clf_name, clf in classifiers.items():
    cv_score = test_classifier(clf, train_preprocessed, train_labels, 
                               vectorizer=bow, text_col='preprocessed_texts')
    print('{}: {}'.format(clf_name, cv_score))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
[0.60750145 0.72474747 0.69714574 0.63225371 0.60215601]
Naive Bayes: 0.6527608791811271
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
[0.56468209 0.58368056 0.57806452 0.60507246 0.52813853]
Support Vector Machine: 0.5719276299198996


/anaconda3/envs/uiuc/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/anaconda3/envs/uiuc/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
[0.56363636 0.55510204 0.61438679 0.66009271 0.65766913]
Logistic Regression: 0.6101774069626231
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=26, verbose=0,
                       warm_start=False)
[0.6613695  0.59050546 0.63050847 

### 2.3 TFIDF - No Preprocessing

In [325]:
%%time
for clf_name, clf in classifiers.items():
    cv_score = test_classifier(clf, train, train_labels, 
                               vectorizer=tfidf, text_col='text')
    print('{}: {}'.format(clf_name, cv_score))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
[0.6447205  0.72474747 0.668357   0.66947439 0.60507246]
Naive Bayes: 0.66247436538252
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
[0.63527851 0.72474747 0.66902834 0.62271    0.62594372]
Support Vector Machine: 0.6555416107222085
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
[0.63588216 0.69714574 0.66902834 0.63274933 0.63916476]
Logistic Regression: 0.6547940650277616
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
        

### 2.3 TFIDF - Preprocessing

In [326]:
%%time
for clf_name, clf in classifiers.items():
    cv_score = test_classifier(clf, train_preprocessed, train_labels, 
                               vectorizer=tfidf, text_col='preprocessed_texts')
    print('{}: {}'.format(clf_name, cv_score))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
[0.62650104 0.72474747 0.67792317 0.68783693 0.60507246]
Naive Bayes: 0.6644162150542321
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
[0.61704244 0.72474747 0.65951878 0.59547908 0.62594372]
Support Vector Machine: 0.6445463003313365
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
[0.65408805 0.72456199 0.69683944 0.60497261 0.61754386]
Logistic Regression: 0.6596011913654566
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
      

## 3. Additional Attempts
Word Embeddings: Gensim word2vec
Ensemble: mlens

To try in future:
* Undersampling
* fasttext
* BERT
* Deep Learning

### 3a. word2vec embeddings

In [101]:
%%time
# train word2vec on all the texts - both training and test set
# we're not using test labels, just texts so this is fine
num_workers = multiprocessing.cpu_count()
print(num_workers)

word_model = Word2Vec(preprocessed_texts, 
                      size=300, window=5, min_count=3, 
                      workers=num_workers)
w2v = {w: vec for w, vec in zip(word_model.wv.index2word, word_model.wv.syn0)}
mean_vec_tr = MeanEmbeddingVectorizer(word_model)
w2v_embeddings = mean_vec_tr.transform(preprocessed_texts)

12


/anaconda3/envs/uiuc/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  if __name__ == '__main__':


CPU times: user 2min 28s, sys: 1.21 s, total: 2min 29s
Wall time: 42.9 s


In [133]:
w2v_train = w2v_embeddings[:546]
w2v_test = w2v_embeddings[546:]
print(w2v_train.shape)
print(w2v_test.shape)
      
print(train_df.shape)
print(test_df.shape)

(546, 300)
(12753, 300)
(546, 8)
(12753, 8)


In [135]:
%%time
w2v_train = w2v_embeddings[:546]
clf = svm.SVC()# XGBClassifier(n_estimators=500, n_jobs=-1)
scores = cross_val_score(clf, w2v_train, train_labels, cv=5, scoring= 'f1')
print(scores)
print("Average F1-Score: %0.5f" % np.average(scores))

[0.57425743 0.66       0.67272727 0.64       0.5625    ]
Average F1-Score: 0.62190
CPU times: user 356 ms, sys: 1.86 ms, total: 358 ms
Wall time: 357 ms


In [137]:
%%time
w2v_train = w2v_embeddings[:546]
params = {'max_depth': 6, 
         'eta': 0.3, 
         'objective': 'binary:logistic', 
         'subsample':0.8, 
         'n_estimators': 500,
         'scale_pos_weight': 2,
         'eval_metric': 'auc',
         'n_jobs': -1}
clf = XGBClassifier(n_estimators=500, 
                    max_depth=5, 
                    learning_rate=0.2, 
                    objective='binary:logistic',
                    scale_pos_weight=2,
                    n_jobs=-1,
                    random_state=SEED)
scores = cross_val_score(clf, w2v_train, train_labels, cv=5, scoring= 'f1')
print(scores)
print("Average F1-Score: %0.5f" % np.average(scores))

[0.61946903 0.65517241 0.61538462 0.65       0.56074766]
Average F1-Score: 0.62015
CPU times: user 11.7 s, sys: 18.6 ms, total: 11.7 s
Wall time: 11.7 s


### 3b. Ensemble Using mlens
* https://mlens.readthedocs.io/en/0.1.x/ensemble_tutorial/#ensemble-model-selection


In [316]:
from mlens.ensemble import SuperLearner
from sklearn.metrics import f1_score

def f1(y, p): return f1_score(y, p, average='macro')

In [317]:
# --- Build ---

# Passing a scoring function will create cv scores during fitting
# the scorer should be a simple function accepting to vectors and returning a scalar
ensemble = SuperLearner(scorer=f1, random_state=SEED)

# Build the first layer
layer1_classifiers = [MultinomialNB(),
                      SVC(probability=False),
                      LogisticRegression(),
                      XGBClassifier(n_estimators=500, 
                            max_depth=5, 
                            learning_rate=0.2, 
                            objective='binary:logistic',
                            scale_pos_weight=2,
                            n_jobs=-1,
                            random_state=SEED),
                      RandomForestClassifier(random_state=SEED, n_estimators=500, n_jobs=-1)]
ensemble.add(layer1_classifiers)

# Attach the final meta estimator
ensemble.add_meta(RandomForestClassifier(random_state=SEED, n_estimators=500, n_jobs=-1))


SuperLearner(array_check=None, backend=None, folds=2,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=4917, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=2, raise_on_ex...108400>)],
   n_jobs=-1, name='group-38', raise_on_exception=True, transformers=[])],
   verbose=0)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=26, sample_size=20,
       scorer=<function f1 at 0x1a69108400>, shuffle=False, verbose=False)

In [318]:
%%time
pipeline = Pipeline([
    ('union', ColumnTransformer(
        [('cuisines_offered', CountVectorizer(min_df=10), 'cuisines_offered'),
         ('zipcode', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['zipcode']),
         ('num_reviews', CountVectorizer(max_df=7, token_pattern='\d+'), 'num_reviews'),
         ('avg_rating', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['avg_rating']),
         ('text', TfidfVectorizer(
                    stop_words='english',
                    strip_accents='unicode',
                    min_df=3,
                    max_df=0.5,
                    ngram_range=(1, 3),
                    max_features=500), 'preprocessed_texts')],
        remainder='passthrough',
    )),
    ('clf', ensemble)
], verbose=False)

# pipeline.fit(X_train, y_train)
# y_pred = pipeline.predict(X_test)
# score = metrics.f1_score(y_test, y_pred)
scores = cross_val_score(pipeline, train_preprocessed, train_labels, cv=5, scoring= 'f1')
print(scores)
print("Average F1-Score: %0.5f" % np.average(scores))

[0.58715596 0.69565217 0.63247863 0.63063063 0.57446809]
Average F1-Score: 0.62408
CPU times: user 1min 19s, sys: 11.7 s, total: 1min 31s
Wall time: 46.4 s


In [175]:
%%time
pipeline.fit(train, train_labels)
y_pred = pipeline.predict(test)

CPU times: user 39.2 s, sys: 1.33 s, total: 40.5 s
Wall time: 31.1 s


In [180]:
submit_path ='./submissions/submission7_mlens1.txt'
create_submission(y_pred, submit_path)

## 4. Submission

In [179]:
def create_submission(y_pred, filepath):
    with open(filepath, 'w') as f:
        f.write('jc26\n')
        for label in y_pred:
            f.write(str(int(label)) + '\n')

In [335]:
%%time
pipeline = Pipeline([
    ('preprocess', ColumnTransformer(
        [('cuisines_offered', CountVectorizer(min_df=10), 'cuisines_offered'),
         ('zipcode', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['zipcode']),
         ('num_reviews', CountVectorizer(max_df=7, token_pattern='\d+'), 'num_reviews'),
         ('avg_rating', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['avg_rating']),
         ('text', TfidfVectorizer(
                    stop_words='english',
                    strip_accents='unicode',
                    min_df=3,
                    max_df=0.5,
                    ngram_range=(1, 3),
                    max_features=500), 'preprocessed_texts')],
        remainder='passthrough',
    )),
    ('clf', MultinomialNB())
], verbose=False)

pipeline.fit(train_preprocessed, train_labels)
y_pred = pipeline.predict(test_preprocessed)
# score = metrics.f1_score(y_test, y_pred)
# scores = cross_val_score(pipeline, train_preprocessed, train_labels, cv=5, scoring= 'f1_macro')
# print(scores)
# print("Average F1-Score: %0.5f" % np.average(scores))

CPU times: user 16.3 s, sys: 227 ms, total: 16.5 s
Wall time: 16.6 s


In [336]:
submit_path ='./submissions/submission10_SVM_preprocessed_countVectorizer.txt'
create_submission(y_pred, submit_path)

### Double Check Before Submitting!!!
And just run the follwing cell

In [337]:
submit_path

'./submissions/submission9_NB_preprocessed_countVectorizer.txt'

In [338]:
!python submit.py jc26 {submit_path}

Submission completed successfully!


## Discussion

### Method Comparison
Tried two or more text representation techniques and two or more learning algorithms and used the additional features; The comparison gives an insight on why some methods perform better than others.

* Text Representation: How text is represented as features (for example, unigrams is one technique, bigrams is another)
* Learning Algorithm: The classification algorithm used (for example, SVM, Naive Bayes, Logistic Regression, ... )


### Best Performing Method
* What toolkit was used?
* How was text preprocessed? (i.e., stopword removal, stemming, or any data cleaning technique)
* How was text represented as features?
* What was the learning algorithm used?

## References
* https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer.html#sphx-glr-auto-examples-compose-plot-column-transformer-py
* https://towardsdatascience.com/nlp-performance-of-different-word-embeddings-on-text-classification-de648c6262b
